In [28]:
import os
import numpy as np

from IPython.core.debugger import Tracer

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam


class GAN(object):
    """ Generative Adversarial Network class """
    def __init__(self, width=28, height=28, channels=1):

        self.width = width
        self.height = height
        self.channels = channels

        self.shape = (self.width, self.height, self.channels)

        self.optimizer = Adam(lr=0.0002, beta_1=0.5, decay=8e-8)

        self.G = self.__generator()
        self.G.compile(loss='binary_crossentropy', optimizer=self.optimizer)

        self.D = self.__discriminator()
        self.D.compile(loss='binary_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])

        self.stacked_generator_discriminator = self.__stacked_generator_discriminator()

        self.stacked_generator_discriminator.compile(loss='binary_crossentropy', optimizer=self.optimizer)


    def __generator(self):
        """ Declare generator """

        model = Sequential()
        model.add(Dense(256, input_shape=(100,)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.width  * self.height * self.channels, activation='tanh'))
        model.add(Reshape((self.width, self.height, self.channels)))

        return model

    def __discriminator(self):
        """ Declare discriminator """

        model = Sequential()
        model.add(Flatten(input_shape=self.shape))
        model.add(Dense((self.width * self.height * self.channels), input_shape=self.shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(np.int64((self.width * self.height * self.channels)/2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        return model

    def __stacked_generator_discriminator(self):

        self.D.trainable = False

        model = Sequential()
        model.add(self.G)
        model.add(self.D)

        return model

    def train(self, X_train, epochs=20000, batch = 32, save_interval = 100):

        for cnt in range(epochs):

            ## train discriminator
            random_index = np.random.randint(0, len(X_train) - np.int64(batch/2))
            legit_images = X_train[random_index : random_index + np.int64(batch/2)].reshape(np.int64(batch/2), self.width, self.height, self.channels)

            gen_noise = np.random.normal(0, 1, (np.int64(batch/2), 100))
            syntetic_images = self.G.predict(gen_noise)

            x_combined_batch = np.concatenate((legit_images, syntetic_images))
            y_combined_batch = np.concatenate((np.ones((np.int64(batch/2), 1)), np.zeros((np.int64(batch/2), 1))))

            d_loss = self.D.train_on_batch(x_combined_batch, y_combined_batch)


            # train generator

            noise = np.random.normal(0, 1, (batch, 100))
            y_mislabled = np.ones((batch, 1))

            g_loss = self.stacked_generator_discriminator.train_on_batch(noise, y_mislabled)

            print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

            if cnt % save_interval == 0:
                self.plot_images(save2file=True, step=cnt)


    def plot_images(self, save2file=False, samples=16, step=0):
        ''' Plot and generated images '''
        if not os.path.exists("./images"):
            os.makedirs("./images")
        filename = "./images/mnist_%d.png" % step
        noise = np.random.normal(0, 1, (samples, 100))

        images = self.G.predict(noise)

        plt.figure(figsize=(10, 10))

        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.height, self.width])
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()

        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()



In [29]:
if __name__ == '__main__':
    (X_train, _), (_, _) = mnist.load_data()
    
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = np.expand_dims(X_train, axis = 3)
    
    gan = GAN()
    gan.train(X_train)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 784)               615440    
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 784)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 392)               307720    
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 392)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 393       
Total params: 923,553
Trainable params: 923,553
Non-trainable params: 0
_________________________________________________________________
epoc

epoch: 101, [Discriminator :: d_loss: 0.008834], [ Generator :: loss: 5.235103]
epoch: 102, [Discriminator :: d_loss: 0.005410], [ Generator :: loss: 5.061078]
epoch: 103, [Discriminator :: d_loss: 0.015551], [ Generator :: loss: 5.306393]
epoch: 104, [Discriminator :: d_loss: 0.006013], [ Generator :: loss: 5.274885]
epoch: 105, [Discriminator :: d_loss: 0.007648], [ Generator :: loss: 5.283916]
epoch: 106, [Discriminator :: d_loss: 0.005071], [ Generator :: loss: 5.210989]
epoch: 107, [Discriminator :: d_loss: 0.013253], [ Generator :: loss: 5.284568]
epoch: 108, [Discriminator :: d_loss: 0.005180], [ Generator :: loss: 5.156510]
epoch: 109, [Discriminator :: d_loss: 0.006616], [ Generator :: loss: 5.131604]
epoch: 110, [Discriminator :: d_loss: 0.007337], [ Generator :: loss: 5.113583]
epoch: 111, [Discriminator :: d_loss: 0.005587], [ Generator :: loss: 5.093506]
epoch: 112, [Discriminator :: d_loss: 0.010856], [ Generator :: loss: 5.174223]
epoch: 113, [Discriminator :: d_loss: 0.

epoch: 217, [Discriminator :: d_loss: 1.263194], [ Generator :: loss: 0.471904]
epoch: 218, [Discriminator :: d_loss: 1.036532], [ Generator :: loss: 2.079058]
epoch: 219, [Discriminator :: d_loss: 0.204380], [ Generator :: loss: 5.060790]
epoch: 220, [Discriminator :: d_loss: 0.937429], [ Generator :: loss: 0.829539]
epoch: 221, [Discriminator :: d_loss: 0.522674], [ Generator :: loss: 0.584798]
epoch: 222, [Discriminator :: d_loss: 0.629059], [ Generator :: loss: 1.620822]
epoch: 223, [Discriminator :: d_loss: 0.245022], [ Generator :: loss: 3.511215]
epoch: 224, [Discriminator :: d_loss: 0.536433], [ Generator :: loss: 1.422654]
epoch: 225, [Discriminator :: d_loss: 0.356943], [ Generator :: loss: 1.334450]
epoch: 226, [Discriminator :: d_loss: 0.258641], [ Generator :: loss: 1.919647]
epoch: 227, [Discriminator :: d_loss: 0.256625], [ Generator :: loss: 1.845612]
epoch: 228, [Discriminator :: d_loss: 0.314645], [ Generator :: loss: 1.534050]
epoch: 229, [Discriminator :: d_loss: 0.

epoch: 335, [Discriminator :: d_loss: 0.549324], [ Generator :: loss: 0.829743]
epoch: 336, [Discriminator :: d_loss: 0.580490], [ Generator :: loss: 0.480018]
epoch: 337, [Discriminator :: d_loss: 0.674302], [ Generator :: loss: 0.934063]
epoch: 338, [Discriminator :: d_loss: 0.639754], [ Generator :: loss: 0.513109]
epoch: 339, [Discriminator :: d_loss: 0.649341], [ Generator :: loss: 1.184030]
epoch: 340, [Discriminator :: d_loss: 0.688191], [ Generator :: loss: 0.471781]
epoch: 341, [Discriminator :: d_loss: 0.599375], [ Generator :: loss: 0.763469]
epoch: 342, [Discriminator :: d_loss: 0.535154], [ Generator :: loss: 0.963199]
epoch: 343, [Discriminator :: d_loss: 0.621849], [ Generator :: loss: 0.531794]
epoch: 344, [Discriminator :: d_loss: 0.644311], [ Generator :: loss: 0.656908]
epoch: 345, [Discriminator :: d_loss: 0.586105], [ Generator :: loss: 1.185977]
epoch: 346, [Discriminator :: d_loss: 0.702294], [ Generator :: loss: 0.475049]
epoch: 347, [Discriminator :: d_loss: 0.

epoch: 451, [Discriminator :: d_loss: 0.607542], [ Generator :: loss: 0.732086]
epoch: 452, [Discriminator :: d_loss: 0.578905], [ Generator :: loss: 0.795439]
epoch: 453, [Discriminator :: d_loss: 0.581270], [ Generator :: loss: 0.866432]
epoch: 454, [Discriminator :: d_loss: 0.562976], [ Generator :: loss: 0.498891]
epoch: 455, [Discriminator :: d_loss: 0.674201], [ Generator :: loss: 0.792686]
epoch: 456, [Discriminator :: d_loss: 0.658367], [ Generator :: loss: 0.917117]
epoch: 457, [Discriminator :: d_loss: 0.614225], [ Generator :: loss: 0.551126]
epoch: 458, [Discriminator :: d_loss: 0.628098], [ Generator :: loss: 0.766905]
epoch: 459, [Discriminator :: d_loss: 0.615355], [ Generator :: loss: 0.798027]
epoch: 460, [Discriminator :: d_loss: 0.626189], [ Generator :: loss: 0.590084]
epoch: 461, [Discriminator :: d_loss: 0.634571], [ Generator :: loss: 0.927678]
epoch: 462, [Discriminator :: d_loss: 0.608284], [ Generator :: loss: 0.802262]
epoch: 463, [Discriminator :: d_loss: 0.

...

epoch: 19501, [Discriminator :: d_loss: 0.580813], [ Generator :: loss: 0.694567]
epoch: 19502, [Discriminator :: d_loss: 0.571929], [ Generator :: loss: 0.816381]
epoch: 19503, [Discriminator :: d_loss: 0.641999], [ Generator :: loss: 0.894183]
epoch: 19504, [Discriminator :: d_loss: 0.601689], [ Generator :: loss: 1.085536]
epoch: 19505, [Discriminator :: d_loss: 0.715446], [ Generator :: loss: 0.729337]
epoch: 19506, [Discriminator :: d_loss: 0.715111], [ Generator :: loss: 0.601851]
epoch: 19507, [Discriminator :: d_loss: 0.570024], [ Generator :: loss: 0.776098]
epoch: 19508, [Discriminator :: d_loss: 0.690093], [ Generator :: loss: 0.911509]
epoch: 19509, [Discriminator :: d_loss: 0.638442], [ Generator :: loss: 0.836043]
epoch: 19510, [Discriminator :: d_loss: 0.628373], [ Generator :: loss: 0.844755]
epoch: 19511, [Discriminator :: d_loss: 0.619666], [ Generator :: loss: 0.785382]
epoch: 19512, [Discriminator :: d_loss: 0.681905], [ Generator :: loss: 0.916397]
epoch: 19513, [D

epoch: 19601, [Discriminator :: d_loss: 0.590357], [ Generator :: loss: 0.833230]
epoch: 19602, [Discriminator :: d_loss: 0.605505], [ Generator :: loss: 0.984833]
epoch: 19603, [Discriminator :: d_loss: 0.597548], [ Generator :: loss: 1.033495]
epoch: 19604, [Discriminator :: d_loss: 0.662148], [ Generator :: loss: 0.818049]
epoch: 19605, [Discriminator :: d_loss: 0.641810], [ Generator :: loss: 0.747686]
epoch: 19606, [Discriminator :: d_loss: 0.750059], [ Generator :: loss: 0.987651]
epoch: 19607, [Discriminator :: d_loss: 0.601228], [ Generator :: loss: 1.191734]
epoch: 19608, [Discriminator :: d_loss: 0.598684], [ Generator :: loss: 0.985297]
epoch: 19609, [Discriminator :: d_loss: 0.628228], [ Generator :: loss: 0.835279]
epoch: 19610, [Discriminator :: d_loss: 0.692677], [ Generator :: loss: 0.633866]
epoch: 19611, [Discriminator :: d_loss: 0.873675], [ Generator :: loss: 0.679918]
epoch: 19612, [Discriminator :: d_loss: 0.673126], [ Generator :: loss: 0.804083]
epoch: 19613, [D

epoch: 19701, [Discriminator :: d_loss: 0.669785], [ Generator :: loss: 0.844240]
epoch: 19702, [Discriminator :: d_loss: 0.677441], [ Generator :: loss: 0.820313]
epoch: 19703, [Discriminator :: d_loss: 0.603728], [ Generator :: loss: 0.873730]
epoch: 19704, [Discriminator :: d_loss: 0.759561], [ Generator :: loss: 0.857707]
epoch: 19705, [Discriminator :: d_loss: 0.702181], [ Generator :: loss: 0.760396]
epoch: 19706, [Discriminator :: d_loss: 0.559707], [ Generator :: loss: 0.923190]
epoch: 19707, [Discriminator :: d_loss: 0.616704], [ Generator :: loss: 0.743716]
epoch: 19708, [Discriminator :: d_loss: 0.586616], [ Generator :: loss: 0.826073]
epoch: 19709, [Discriminator :: d_loss: 0.672711], [ Generator :: loss: 1.038617]
epoch: 19710, [Discriminator :: d_loss: 0.681632], [ Generator :: loss: 0.895581]
epoch: 19711, [Discriminator :: d_loss: 0.574195], [ Generator :: loss: 0.832497]
epoch: 19712, [Discriminator :: d_loss: 0.724035], [ Generator :: loss: 0.745594]
epoch: 19713, [D

epoch: 19801, [Discriminator :: d_loss: 0.632204], [ Generator :: loss: 0.919613]
epoch: 19802, [Discriminator :: d_loss: 0.701571], [ Generator :: loss: 0.804906]
epoch: 19803, [Discriminator :: d_loss: 0.760035], [ Generator :: loss: 0.818254]
epoch: 19804, [Discriminator :: d_loss: 0.667314], [ Generator :: loss: 0.705480]
epoch: 19805, [Discriminator :: d_loss: 0.606511], [ Generator :: loss: 0.801756]
epoch: 19806, [Discriminator :: d_loss: 0.731657], [ Generator :: loss: 0.936450]
epoch: 19807, [Discriminator :: d_loss: 0.502529], [ Generator :: loss: 1.045531]
epoch: 19808, [Discriminator :: d_loss: 0.613615], [ Generator :: loss: 0.753933]
epoch: 19809, [Discriminator :: d_loss: 0.691317], [ Generator :: loss: 0.833759]
epoch: 19810, [Discriminator :: d_loss: 0.685303], [ Generator :: loss: 0.829856]
epoch: 19811, [Discriminator :: d_loss: 0.741177], [ Generator :: loss: 0.844134]
epoch: 19812, [Discriminator :: d_loss: 0.583764], [ Generator :: loss: 0.874467]
epoch: 19813, [D

epoch: 19901, [Discriminator :: d_loss: 0.697110], [ Generator :: loss: 0.853552]
epoch: 19902, [Discriminator :: d_loss: 0.592163], [ Generator :: loss: 0.926459]
epoch: 19903, [Discriminator :: d_loss: 0.580598], [ Generator :: loss: 1.036944]
epoch: 19904, [Discriminator :: d_loss: 0.666009], [ Generator :: loss: 0.760774]
epoch: 19905, [Discriminator :: d_loss: 0.714423], [ Generator :: loss: 0.695614]
epoch: 19906, [Discriminator :: d_loss: 0.689764], [ Generator :: loss: 0.761420]
epoch: 19907, [Discriminator :: d_loss: 0.573652], [ Generator :: loss: 0.791068]
epoch: 19908, [Discriminator :: d_loss: 0.719907], [ Generator :: loss: 0.704424]
epoch: 19909, [Discriminator :: d_loss: 0.622396], [ Generator :: loss: 0.798802]
epoch: 19910, [Discriminator :: d_loss: 0.608270], [ Generator :: loss: 0.819414]
epoch: 19911, [Discriminator :: d_loss: 0.686238], [ Generator :: loss: 0.866693]
epoch: 19912, [Discriminator :: d_loss: 0.676511], [ Generator :: loss: 0.727894]
epoch: 19913, [D